# CallBack

---

Calbacks, são funções responsáveis para gerar interatividade na aplicação. Essas funções recebem inputs e de acordo com esse ou esses inputs, ele muda os gráficos associados. Eles são associados a ids para garantir que uma determinada ação gere uma mudança no gráfico com o id informado.

## Callback exemplo - 1
### Miltiplos inputs e Multiplo Outputs

In [8]:
!pip install pandas_datareader

In [ ]:
import pandas as pd
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas_datareader.data as web
import datetime



In [ ]:
# https://stooq.com/
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2021, 2, 15)

df = web.DataReader(['SVAC','JNJ','PFE','MRNA','BNTX'],
                    'stooq', start=start, end=end)

df = df.stack().reset_index()

cores = ["black", "red", "blue", "orange", 'yellow']


In [8]:
df.head()

Attributes,Date,Symbols,Close,High,Low,Open,Volume
0,2019-01-02,JNJ,120.210,120.810,118.830,120.580,8110385.0
1,2019-01-02,PFE,39.634,39.863,39.070,39.515,27364131.0
2,2019-01-02,MRNA,15.330,15.450,14.510,14.930,2318094.0
3,2019-01-03,JNJ,118.290,120.710,117.960,120.580,9197569.0
4,2019-01-03,PFE,38.524,39.716,38.442,39.624,29510593.0


In [9]:
df.Symbols.unique()

array(['JNJ', 'PFE', 'MRNA', 'BNTX', 'SVAC'], dtype=object)

In [13]:
df[df["Symbols"].isin(['PFE','BNTX'])]

Attributes,Date,Symbols,Close,High,Low,Open,Volume
1,2019-01-02,PFE,39.634,39.863,39.070,39.515,27364131.0
4,2019-01-03,PFE,38.524,39.716,38.442,39.624,29510593.0
7,2019-01-04,PFE,39.404,39.721,38.721,38.892,28104451.0
10,2019-01-07,PFE,39.616,39.973,39.277,39.470,21737296.0
13,2019-01-08,PFE,39.799,40.046,39.542,39.918,20372429.0
...,...,...,...,...,...,...,...
1997,2021-02-10,BNTX,119.350,120.330,117.100,119.055,1397490.0
2000,2021-02-11,PFE,34.430,34.810,34.280,34.800,30885165.0
2002,2021-02-11,BNTX,118.030,119.930,115.420,119.000,1352840.0
2005,2021-02-12,PFE,34.720,34.950,34.360,34.440,25578646.0


In [5]:
app1 = dash.Dash(__name__)

app1.layout = html.Div(

        [
        
        # dash component 
        html.H1(id = 'output-titulo', children='Valor de Ação - Covid-19',  style={} ),
            
        # dash component 
        dcc.Dropdown( id = 'meu-dropdown', multi = True,
                    options = [{ 'label': x , 'value': x   } for x in sorted(df.Symbols.unique())  ],
                    value =['SVAC','JNJ','PFE','MRNA','BNTX']
                    ,style={'text-align': 'center'}  
                    ),
        # dash component 
        dcc.Graph(id = 'grafico-output', figure ={}),
            
        # dash component 
        dcc.RadioItems(id = 'cores-id', value = 'black', options = [{ 'label': c, 'value': c   } for c in cores]),
        ]

)


In [6]:
@app1.callback(
    [Output(component_id='output-titulo', component_property='style'),
     Output(component_id='grafico-output', component_property='figure')],
    
    [Input(component_id='cores-id', component_property='value'),
    Input(component_id='meu-dropdown', component_property='value')],
   # prevent_initial_call=False # True : Não ativa o callback quando a pagina é atualizada
)

def muda_grafico(titulo_style, dropdown_figure):

    if len(dropdown_figure) == 0:
        return {"color": titulo_style, 'text-align': 'center'}, dash.no_update # faz update em somente um componente quando a página é atualizada - nesse caso a cor e nao o grafico
    else:
        df_copy = df[df["Symbols"].isin(dropdown_figure)]
        fig = px.line(df_copy, x='Date', y='Close', color = 'Symbols')

    return {"color": titulo_style, 'text-align': 'right', 'background-color': 'powderblue'}, fig



In [7]:

if __name__ == "__main__":
    app1.run_server(debug=True, port=2, use_reloader=False)

Dash is running on http://127.0.0.1:2/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


---